In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import mysql.connector
import networkx as nx
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon,MultiPolygon,LineString,MultiLineString,LinearRing,Point
from shapely import symmetric_difference
from shapely.ops import nearest_points
from shapely.wkt import loads
import scipy
import time
import numpy as np
import random
from networkx.drawing.nx_pydot import graphviz_layout
import matplotlib.pyplot as plt
import osmnx as ox
import glob
from itertools import combinations,product
from copy import deepcopy
import os
from lib_contour_lines import *
import mysql.connector


In [2]:

cnx = mysql.connector.connect(user='spirz', 
                              password='this_is_my_PASSWORD_m8',
                              host='localhost',
                              database='dem_from_contours_lines',
                              autocommit=True)
cursor = cnx.cursor(buffered=True,dictionary=True)
contours_lines_table_name='contours_lines'
osm_nodes_table_name="osm_nodes"
osm_edges_table_name="osm_edges"
intersections_table_name="intersections"
2

2

In [3]:
t1=time.time()
place_name='Corse, France'
G_osm=ox.graph_from_place(place_name,network_type='drive')

t2=time.time()
print(t2-t1)


56.739688873291016


In [4]:
DELETE_TABLES=True
if DELETE_TABLES:
    delete_tables(cursor,table_names=["intersections","osm_edges","osm_nodes"])
2

2

In [5]:
CREATE_TABLES=True
if CREATE_TABLES:
    create_tables_v2(cursor,contours_lines_table_name='contours_lines',osm_nodes_table_name="osm_nodes",osm_edges_table_name="osm_edges",intersections_table_name="intersections")
2

2

In [6]:
t1=time.time()
G_osm,osm_crs=preprocess_osm_graph(G_osm)
t2=time.time()
print(t2-t1)

12.665990114212036


In [7]:
ADD_OSM_DATA=True
if ADD_OSM_DATA:
    t1=time.time()
    compute_all_intersections(G_osm,osm_crs,cursor)    
    t2=time.time()
    print(t2-t1)
2

0.000000<=elevation<50.000000 :26.643717 
50.000000<=elevation<100.000000 :25.397292 
100.000000<=elevation<150.000000 :17.586095 
150.000000<=elevation<200.000000 :16.437946 
200.000000<=elevation<250.000000 :16.355820 
250.000000<=elevation<300.000000 :15.279205 
300.000000<=elevation<350.000000 :15.416771 
350.000000<=elevation<400.000000 :15.021826 
400.000000<=elevation<450.000000 :16.461648 
450.000000<=elevation<500.000000 :12.816448 
500.000000<=elevation<550.000000 :10.917080 
550.000000<=elevation<600.000000 :9.135464 
600.000000<=elevation<650.000000 :8.800387 
650.000000<=elevation<700.000000 :7.820996 
700.000000<=elevation<750.000000 :7.153443 
750.000000<=elevation<800.000000 :6.588936 
800.000000<=elevation<850.000000 :6.127492 
850.000000<=elevation<900.000000 :5.563487 
900.000000<=elevation<950.000000 :4.912424 
950.000000<=elevation<1000.000000 :4.487072 
1000.000000<=elevation<1050.000000 :4.095056 
1050.000000<=elevation<1100.000000 :3.895736 
1100.000000<=elevati

2

In [8]:
638.7775819301605

638.7775819301605